# Improved Media Content Search

First, we install the lyricsgenius API, as well as multiprocess to increase speed of data scraping:

In [15]:
!pip install multiprocess
!pip install lyricsgenius

We import the libraries and set a path for our input file of artists:

In [11]:
import json
import csv
import multiprocess
import queue
import logging
import sys

from requests.exceptions import HTTPError, ConnectionError, RequestException
from lyricsgenius import Genius

# OS agnostic
import os 
CSV_PATH = os.path.join(os.path.curdir, 'artists', '10000-MTV-Music-Artists-page-%s.csv')

# 1. Scrape Data

We set up a lyricsgenius token, and use the API to pull the lyrics data for each artist in the dataset for the top 10,000 artists from MTV.

In [12]:
def genius_setup():
    """Set up the Genius API
    """
    token = "EBufquOcw_ts4Y4V7yiddUNyUakTdqCpnMZhiI3XtAScWOntEom8Hj4T87gAV_cA"
    genius = Genius(token, retries=2)

    genius.verbose = False
    genius.remove_section_headers = True
    genius.skip_non_songs = True
    genius.excluded_terms = ["(Remix)", "(Live)"]

    return genius    

"""Global variables used in scraping
"""
# Multiprocessing cores
process_number = int(multiprocess.cpu_count()) * 2

# Data management
final_ = multiprocess.Manager().list()
checked_artists = set()

# Output file
file_name = "song_data_2.csv"


def get_artists(queue):
    """Fetch artists
    """
    for x in range(1,5):
        path = CSV_PATH % str(x)
        with open(path, encoding="UTF-8") as csvfile:
            TopArtists = csv.reader(csvfile)
            
            # Skip header
            next(TopArtists)
            for row in TopArtists:
                artist = row[0]
                # Check if we should skip this artists since we already found the data
                if artist not in checked_artists:
                    queue.put(artist)


# File management
def write_to_csv(data):
    """Write to the CSV file
    
    data: list of dictionaries {artist, song, data}
    """
    global file_name
    
    csv_path = os.path.join(os.path.curdir, 'data', file_name)
    with open(csv_path, 'w') as csv_file: 
        # creating a csv dict writer object 
        print("Entries: {num}".format(num=len(data)))
        keys = data[0].keys()
        writer = csv.DictWriter(csv_file, fieldnames = keys) 
        
        # writing headers (field names) 
        writer.writeheader() 
        
        # writing data rows 
        writer.writerows(data) 
        

def read_csv():
    """Read the CSV file
    """
    global final_, checked_artists, file_name   
    
    csv_path = os.path.join(os.path.curdir, 'data', file_name)
    
    # opening the CSV file
    try:
        with open(csv_path, mode ='r', encoding="UTF-8") as file:   

            # reading the CSV file
            data = csv.DictReader(file)

            for entry in data:
                checked_artists.add(entry["artist"])
                final_.append(entry)
                
        print("Number of artists already found {num}".format(num=len(checked_artists)))
    except FileNotFoundError:
        pass
    

# Run genius search
def search_genius(args):
    """Run our search using the Genius API
    """
    import sys
    from requests.exceptions import RequestException
    artist_queue, num, genius, final_ = args
    
    def log(string):
        print("[{num}] ".format(num=num) + string + "\n", end='')
        sys.stdout.flush()
    
    # Processing
    def clean_data(data):
        cleaned_data = data.replace("\n", "|").replace(",", " ")
        return cleaned_data

    def process_artist(artist):
        artist_dict = artist.to_dict()
        return ""

    def process_song(song):
        lyrics = clean_data(song.lyrics)
        return lyrics

    def build_entry(artist, song, data, columns = ["artist", "song", "data"]):
        entry = {"artist": artist, "song": song, "data": data}
        return entry
    
    log("Starting")
    try:
        while True:
            genius_artist = None
            artist = artist_queue.get()
            if artist is None:
                log("Done")
                return
            log("Remaining: [{queue}]. Searching {artist}".format(queue=artist_queue.qsize(), artist=artist.strip()))
            
            # Pull data for artist from genius
            for x in range(5):
                try:
                    genius_artist = genius.search_artist(artist, per_page=50, get_full_info=False)
                    break
                except RequestException as e:
                    log("HTTPSConnectionPool exception. Attempt {}/5".format(x+1))
                except Exception as e:
                    log("Exception. Attempt {}/3".format(x+1))
            
            log("Finished {artist}".format(num=num, artist=artist.strip()))
            if genius_artist == None:
                log("{artist} not found".format(num=num, artist=artist.strip()))
                continue
                           
            artist_data =  process_artist(genius_artist)
                           
            log("{artist} number of songs: {song_num}".format(num=num, artist=artist.strip(), song_num=len(genius_artist.songs)))
            
            for song in genius_artist.songs:
                song_data = process_song(song)
                
                # Add to final list
                final_.append(build_entry(artist, song.title, song_data))
    
    except Exception as e:
        log("Something went wrong: {error}".format(num=num, error= e))
    
    
def run(multi_core=False):
    """Driver function
    """
    # Setup Genius
    genius = genius_setup()
    
    # Load in any previous data
    print("Reading previous")
    read_csv()
    
    pool = None
    try:  
        if multi_core:
            # multiprocess.log_to_stderr().setLevel(logging.DEBUG)
            print("Multiprocessing with {process_number} processes".format(process_number=process_number))
            
            artist_queue = multiprocess.Manager().Queue()
            get_artists(artist_queue)
            
            for x in range(process_number):
                artist_queue.put(None)
            
            print(artist_queue.qsize())
            # creating processes
            with multiprocess.get_context("spawn").Pool(process_number) as pool:
                args = [(artist_queue, x, genius, final_) for x in range(process_number)]
                pool.map(search_genius, args)
                pool.close()
                pool.join()
            
        else:
            print("Running single core")
            artist_queue = queue.Queue()
            get_artists(artist_queue)
            artist_queue.put(None)
            print(artist_queue.qsize())
            search_genius((artist_queue, 0, genius, final_))

    
    except KeyboardInterrupt:
        if pool:
            pool.close()
            pool.terminate()
            pool.join()
        print("KeyboardInterrupt: Writing results")
    
    finally:
        write_to_csv(list(final_))                       



The cell below kicks off the data scraping (may take 10+ hours)

In [13]:
# run(multi_core=True) 

# 2. Creating the Model

We use TensorFlow to generate text embeddings for our lyric data. First we install Tensorflow and the ANNOY (approximate nearest neighbors) library from Spotify.

In [7]:
!pip3 install tensorflow
!pip3 install tensorflow_hub
!pip3 install annoy
!pip3 install pickle5

In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
# tf.get_logger().setLevel(logging.ERROR)
import tensorflow_hub as hub
from annoy import AnnoyIndex
import pickle5 as pickle
import os
import csv
import numpy as np
import random
from sklearn import random_projection


Here we set global variables to store our Annoy index and our universal counter to keep track of lyric entries

## Grab data

We define several helper functions to load lyric lines documents, and use Tensorflow to get embeddings.

In [15]:
# Define input path
input_file="song_data_2.csv"
input_csv_path = os.path.join(os.path.curdir, 'data', input_file)
csv.field_size_limit(100000000)


def batch_load():
    """Load a batch of 1000 songs - not used unless batching input
    """
    
    batch_sentences = []
    global counter
    
    print("loading batch of songs, starting at song", counter)
    
    for x in range(1000):
        
        document = next(documents)

        data = document['data']
        lines = data.split("|")

        song = document['song']
        artist = document['artist']
        for line in lines:
            ids[counter] = (line, song, artist)
            batch_sentences.append(line)
            counter += 1
        
    return batch_sentences


def get_lines():
    """Get individual lines from the input CSV, to use as the input for embeddings
    """
    songs = []
    
    with open(input_csv_path, mode ='r+', encoding = 'utf-8') as file:   
            datareader = csv.DictReader(file)
            next(datareader)
            for row in datareader:
                data = row['data']
                song = row['song']
                artist = row['artist']
    
                songs.append([data, song, artist])
                    
                # if len(lines) % 100000 == 0:
                #     print("{} lines added".format(len(lines)))
    
    random.shuffle(songs)
    lines = []
    for x in songs:
        data, song, artist = x
        lyrics = data.split("|")

        for lyric in lyrics:
            lines.append([lyric, song, artist])
    
    del songs
    print("total lines added: {}".format(len(lines)))
    
    return lines

In [16]:
# Retrieves the embedding for a batch of sentences 

vector_length = 512

embed_module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")
placeholder = tf.placeholder(dtype=tf.string)
embed = embed_module(placeholder)
session = tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])
transformer = random_projection.GaussianRandomProjection(vector_length)

def get_embeddings(sentences):
    """Gets embeddings for a given line (document)
    """
    
    print("Getting embeddings...")
    embeddings = session.run(embed, feed_dict={placeholder: sentences})
    
    ## Reduction of dimensionality does not work - must use 512 vector length
    # print("Reducing dimensionality...")
    # reduced_embeddings = transformer.fit_transform(embeddings)
    
    return embeddings

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Build the Annoy Index
Finally we build the index for our dataset.

In [17]:
# Set up mapping and counter to keep track of documents
counter = 0
mapping = {}
index_file_name = "annoy_index_512_20_trees"

# Initialize the ANNOY index 
ann = AnnoyIndex(vector_length, metric='angular')


def add_items_to_index(batch, embeddings):
    """Adds items to an ANNOY index
    
    sentences: a list of 
    embeddings: a list of tensorflow embeddings for sentences
    """ 
    global ann, counter, mapping
    
    for line, embed in zip(batch, embeddings):
        ann.add_item(counter, embed)
        mapping[counter] = line
        counter +=1  
        

def build_ann_index(batch_size=200000):
    """Constructs the ANNOY index
    """
    print("getting lines from CSV file...")
    lines = get_lines()
    print("lines retrieved, getting embeddings...")
    
    # num_lines = 100000 * 10
    lines = lines[:8500000]
    
    ann.on_disk_build(index_file_name)

    # get the embeddings in batches - 1/50th of data set to test
    # for x in range(0, num_lines, batch_size):
    while len(lines) > 0:
        
        print("Operating on lines {} - {}".format(counter, counter + batch_size))
        start = 0
        if batch_size >= len(lines):
            end = len(lines)
        else:
            end = batch_size
        
        batch = lines[start:end]
        
        lyrics = [x[0] for x in batch]
        embeddings = get_embeddings(lyrics)
        add_items_to_index(batch, embeddings)
               
        del embeddings, lyrics
        del lines[:end]
        
        print("{} left".format(len(lines)))


The below cell constructs the index.

In [ ]:
try:
    build_ann_index()
except KeyboardInterrupt:
    print("KeyboardInterrupt")
finally:
    print("Building index...")
    ann.build(10)
    ann.unload()
    
    with open(index_file_name + '.mapping', 'wb') as handle:
        pickle.dump(mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    print('mapping saved')

# 3. Load the Model 

Next, we load our saved model and use it for predictions.

In [18]:
ann = AnnoyIndex(vector_length, metric='angular')
ann.load(index_file_name, prefault=True)
print('annoy index loaded.')
with open(index_file_name + '.mapping', 'rb') as handle:
    mapping = pickle.load(handle)
print('mapping file loaded.')

annoy index loaded.
mapping file loaded.


And finally, use a sample query to test the performance of our retreival system:

In [19]:
input_sentence = "Getting an email on my iPhone"

print("Getting query embeddings...")
query_embeddings = get_embeddings([input_sentence])[0]

# Return 10 nearest neighbors
print("Getting nearest neighbors...")
nns = ann.get_nns_by_vector(query_embeddings, 10, include_distances=False)

print("Closest: ")
for idx, item in enumerate(nns):
    print("{}. {} - {}:".format(idx+1, mapping[item][1], mapping[item][2]))
    for x in range(item-3, item+3):
        if x == item:
            print("==== {} ====".format(mapping[x][0]))
        else:
            print("     {}     ".format(mapping[x][0]))
    
    print("\n")


Getting query embeddings...
Getting embeddings...
Getting nearest neighbors...
Closest: 
1. Keep on Shinin’ -  Future :
     I'm turnin' up  you go to sleep     
     I whipped me up that V12     
     I pull up in that V12     
==== I got a hundred bands in my email ====
     I'm on the yacht  no sea shells     
     On tough sand I touch bands     


2. Boost Mobile (Freestyle) -  Young Roddy :
     Alright  let me stop fuckin' around  hold this down for my set     
     Jet Life  ever since Route The Ruler  they been calling me the truth     
     I use to have a hooptie  til Spitta loaned me his coupe     
==== I finally got an iPhone  use to be team boost ====
     But it made me hella money  boy my shoe box the proof     
     I grew up in the hood where nigga's reppin' soowoop     


3. Funkmaster Flex Freestyle (2012) -  Meek Mill :
     Got a couple of Rovers  had a drop one Mercedes     
     I know these suckers don’t like me  I'm probably poppin' they lady     
     I got 1

# 4. Interactive Application

We need ipywidgets to display interactive widgets:

In [4]:
!pip install ipywidgets

First we set up our widgets which can be used to test per-query and make evaluation of the model easier.

In [20]:
import ipywidgets as widgets

query = widgets.Text(
        value='Sitting in my truck',
        description='Input query:')

button = widgets.Button(description='Submit')

slider = widgets.IntSlider(
         value=5,
         min=0,
         max=20,
         step=1,
         description='# of results:')

def on_click(_):
    with query:
        clear_output()
        print(query.value)
        

interact = widgets.TwoByTwoLayout(top_left=query,
                       bottom_left=slider)

interact

TwoByTwoLayout(children=(Text(value='Sitting in my truck', description='Input query:', layout=Layout(grid_area…

Now we run the query defined above, looking for the number of results requested.

In [25]:
user_query = query.value
print("running query...")

query_embeddings = get_embeddings([user_query])[0]

# Return X nearest neighbors
nns = ann.get_nns_by_vector(query_embeddings, slider.value, include_distances=False)

print("Top {} results for \'{}\'\n".format(slider.value, query.value))
for idx, item in enumerate(nns):
    print("{}. {} - {}:".format(idx+1, mapping[item][1], mapping[item][2]))
    for x in range(item-3, item+3):
        if x == item:
            print("==== {} ====".format(mapping[x][0]))
        else:
            print("     {}     ".format(mapping[x][0]))
    
    print("\n")


running query...
Getting embeddings...
Top 5 results for 'Sitting in my truck'

1. Swamp River Days -  John Fogerty :
     Two-lane     
     Shinin' in the july dust     
     Heat risin' off the road     
==== Out in front of my truck ====
     Pulled my pick-up over the hill     
     Down the slide gravel through the cat-tails     


2. Next To You, Next To Me -  Rascal Flatts :
     Next to you  next to me     
     Baby  next to you  next to me     
          
==== Riding down the road in my pick-up truck ====
     You better be ready ’cause I am picking you up     
     With a full moon shining and a little bit a’ luck     


3. Seven -  Boondox :
     They won't leave me alone     
     Tell me snatch this bitch up by hair     
     And drag her home     
==== Over my shoulder in the back of a pickup truck ====
     Can't wait to get home hold her     
     Bleed her then chop her up     


4. First Few Desperate Hours -  Mountain :
     Bad luck comes in from Tampa     
     B